In [1]:
cd 'OneDrive - Intel Corporation\Documents\research_outside\geometricLWE\sage_notebook'

/home/sage/OneDrive - Intel Corporation/Documents/research_outside/geometricLWE/sage_notebook


In [1]:
load("../framework/instance_gen.sage")
load("../framework/geometry_twist.sage")
load("../framework/geometry.sage")
load("optimize_r.sage")
demo = False
from scipy.io import loadmat
import random

In [3]:
# n = 352
# m = 352
# q = 2 ** 11
# frodo_distribution = [22528, 15616, 5120, 768]
# D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
# temp_vec = vec([draw_from_distribution(D_s) for _ in range(n)])

In [48]:
t = matrix([[1,2],[2,3]])

In [50]:
t_temp = matrix(t); t

[1 2]
[2 3]

In [6]:
load("../framework/geometry.sage")
load("../framework/instance_gen.sage")


"""  Example
Uncomment the following to get an example
of the detailed computation (without redundancy)
"""
upper_bound_as_prob = True
to_print = True
# demo = True
# logging("--- Demonstration mode (no redundancy of the attacks) ---")

for params in ['CCS2']: # [ 'CCS1', 'CCS2', 'CCS3', 'CCS4', 'NIST1', 'NIST2']:
    logging("Set of parameters: " + params)

    if params == 'NIST1':
        # NIST1 FRODOKEM-640
        n = 640
        m = 640
        q = 2**15
        frodo_distribution = [9456, 8857, 7280, 5249, 3321,
                              1844, 898, 384, 144, 47, 13, 3]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        # We used the following seeds for generating Bos et al. data
        # These seeds were generated with the matlab code genValues.m
        sca_seeds = [42, 72, 163, 175, 301, 320, 335, 406, 430, 445]
        param = 4

    elif params == 'NIST2':
        # NIST2 FRODOKEM-976
        n = 976
        m = 976
        q = 65536
        frodo_distribution = [11278, 10277, 7774, 4882, 2545, 1101,
                              396, 118, 29, 6, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [74, 324, 337, 425, 543, 1595, 1707, 2026, 2075, 2707]
        param = 5

    elif params == 'CCS1':
        n = 352
        m = 352
        q = 2 ** 11
        frodo_distribution = [22528, 15616, 5120, 768]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        param = 0

    elif params == 'CCS2':
        n = 592
        m = 592
        q = 2 ** 12
        frodo_distribution = [25120, 15840, 3968, 384, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 3, 4, 5, 6, 7, 8, 9, 14, 16]
        param = 1


    elif params == 'CCS3':
        n = 752
        m = 752
        q = 2 ** 15
        frodo_distribution = [19296, 14704, 6496, 1664, 240, 16]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [2, 4, 5, 7, 9, 10, 12, 13, 14, 15]
        param = 2

    elif params == 'CCS4':
        n = 864
        m = 864
        q = 2 ** 15
        frodo_distribution = [19304, 14700, 6490, 1659, 245, 21, 1]
        D_s = get_distribution_from_table(frodo_distribution, 2 ** 16)
        sca_seeds = [256, 393, 509, 630, 637, 652, 665, 1202, 1264, 1387]
        param = 3


    """  Original Security   """

    A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                               n,
                                               q, m, D_s, D_s, verbosity=0)
    dbdd.integrate_q_vectors(q, indices=range(n, n + m))
    (beta, _) = dbdd.estimate_attack()
    logging("Attack without hints:  %3.2f bikz" % beta, style="HEADER")

    """  Refined Side channel attack  """

    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]


    if demo:
        A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                  n,
                                                  q, m, D_s, D_s, verbosity=2)
        measured = [simu_measured(dbdd.u[0, m + i], measures) for i in range(n)]
        if upper_bound_as_prob:
            estimate_SCA_prob(50, dbdd, measured, 0.5, D_s, D_s)
        else:
            estimate_SCA_num(50, dbdd, measured, 175, D_s, D_s)
    else:
        """  Averaging
        The following averages the measures to get accurate data
        for the paper. The averaging mode is quite long.
        """
        nb_tests_per_params = 1

        # Chosen values for the number of guesses
        if upper_bound_as_prob:
            max_guesses = 0.5
        else:
            if params == 'CCS1':
                max_guesses = 175 + 100
            elif params == 'CCS2':
                max_guesses = 300 + 150
            elif params == 'CCS3':
                max_guesses = 250 + 125
            elif params == 'CCS4':
                max_guesses = 250 + 125
            elif params == 'NIST1':
                max_guesses = 100 + 75
            elif params == 'NIST2':
                max_guesses = 250 + 125

        beta = 0
        beta_hybrid = 0
        proba_success = 0
        guesses_average = 0
        beta_hybrid_before_q = 0
        for i in range(nb_tests_per_params):
            A, b, dbdd = initialize_from_LWE_instance(DBDD_predict_diag,
                                                      n,
                                                      q, m, D_s, D_s, verbosity=0)
            measured = [simu_measured(dbdd.u[0, m + i], measures) for i in range(n)]
            if upper_bound_as_prob:
                b, b_hybrid, p_success, guesses, beta_before_q = estimate_SCA_prob(None, dbdd,
                                                  measured, max_guesses, D_s, to_print)
            else:
                b, b_hybrid, p_success, guesses = estimate_SCA_num(None, dbdd,
                                                  measured, max_guesses, D_s)
            beta += b
            beta_hybrid += b_hybrid
            proba_success += p_success
            guesses_average += guesses
            beta_hybrid_before_q += beta_before_q

        beta /= nb_tests_per_params
        beta_hybrid /= nb_tests_per_params
        proba_success /= nb_tests_per_params
        guesses_average /= nb_tests_per_params
        beta_hybrid_before_q /= nb_tests_per_params
        
        logging("Attack with hints: %3.2f bikz" % beta, style="HEADER")
        logging("Attack with hints and guess before q_vectors: %3.2f bikz"% beta_hybrid_before_q, style="HEADER")
        logging("Attack with hints and guess: %3.2f bikz"% beta_hybrid, style="HEADER")
        logging("Number of guesses: %4d" % guesses_average, style="HEADER")
        logging("Success probability: %3.2f" %proba_success, style="HEADER")


 Set of parameters: CCS2 
 Attack without hints:  447.85 bikz 
number of elements in intersect_indices 209
n = 592 sigma_1^2 = 1107/1024

 Calculating r_post/int values 
 Maximium: r = 1, p = 0, beta = 48.62458140420393 
 Minimum: r = 1.2667013919640723e-65, p = 384, beta = 1.123127532507952, l = (0.47945940853697094, 0.6420177133727664, True) 
1. ln det(LWE matrix):  -1647.532527505059
1. ln det(SCA matrix) - ln det(LWE matrix) -613.6830539537254
partial ellip_norm of input1 SCA ellip (dim = n): 
0.24935643839102214
full ellip_norm of input1 SCA ellip (dim = n + m + 1, should be <=1): 
0.678869217950156
partial ellip_norm of input1 LWE ellip (dim = n): 
0.7177832495945405
full ellip_norm of input1 LWE ellip  (dim = n + m + 1, should be <=1): 
0.9128849751083661
ln det(SCA matrix):  -2261.2155814587845
ln det(LWE matrix):  -1647.532527505059
ln det(SCA matrix) - ln det(LWE matrix) -613.6830539537254
ln det(LWE_opt matrix):  -53.54114623394457
ln det(SCA matrix) - ln det(LWE_opt matrix)

      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real

      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real

 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected

 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected

...

      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real

      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real

      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real Perfect Hint in Hybrid Phase     
      Real

 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected

 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected., Rejected. 
 Redundant hint, Rejected

In [2]:
ln det(SCA matrix):  -2318.2647686438777
ln det(LWE matrix):  -1729.6105034626917
ln det(SCA matrix) - ln det(LWE matrix) -588.654265181186

SyntaxError: invalid syntax (<ipython-input-2-2ccdc031118b>, line 1)

In [13]:
list_var = [1.717884023021924e-09, 0.005175895773673389, 0.08129240215559186, 0.008999127822611586, 6.548946590601012e-05, 1.508780756234512e-06, 0.03447902660078661, 1.758229176415378e-05, 4.5466388026715876e-05, 0.035745362685638105, 0.04290771278497085, 0.00016343342474911537, 0.11985603047514681, 2.440773072046737e-06, 1.0559393100574924e-10, 0.9971166478688085, 0.2499914019238041, 0.13300507702556152, 0.17462069895257804, 2.190686528664651e-10, 0.0002718598815221185, 0.05731045010226543, 0.24369866410850438, 0.013229109180982412, 0.201346048645197, 0.0015886235355990763, 0.22334791164326462, 1.6845697192735543e-07, 0.1587257361660397, 1.3648450829641199e-08, 2.6900921891566615e-06, 3.4557453727291146e-07, 8.335667442752048e-12, 0.2320129192272744, 0.24960664756994655, 2.0108370806133646e-12, 2.4591778855788914e-08, 1.0532132334484245e-05, 0.004648774503319686, 1.9060619749211583e-10, 0.14521350541213995, 1.258062968417014e-09, 6.416319162024257e-08, 0.2270906729529767, 0.12553199646650648, 8.620016658245078e-07, 0.043992463959728, 0.04239590639848845, 2.9617043919571984e-05, 0.00047570551406361794, 0.07323058165375862, 6.7267395412531905e-09, 5.668607252432706e-09, 2.694149727489048e-05, 0.0019169449674065753, 2.206741996579019e-08, 0.047501051705863606, 0.01923032343294917, 3.9875506165465424e-05, 0.15657042421448475, 0.20506029574826168, 6.758672393449017e-10, 0.16874459002417286, 0.24883715489490799, 9.676904251221252e-05, 0.175254735420173, 1.1376642093718914e-05, 0.0011822483129883669, 4.572293596798345e-10, 0.03836442087815972, 0.008539937861197444, 4.236908783886162e-12, 0.06797124176991964, 4.4082218484829596e-08, 0.01827869383667613, 0.0009027371201932866, 0.1609903651342468, 0.05820528052271068, 1.48238666171298e-08, 4.486775534440213e-14, 0.004281549662707995, 1.0800494261165028e-06, 0.0006451623221797043, 4.993040093181911e-05, 0.0024920223839599576, 0.0017813096448708876, 2.782622313857175e-05, 1.2896867263013337e-06, 0.16539513442531262, 0.0015526551556277468, 1.1562990398803397e-05, 0.00018193330073690284, 8.394613351886751e-11, 2.8393749477160276e-07, 0.23011305985721342, 2.4838655918743915e-08, 0.037302704632982575, 1.99567898231251e-09, 0.23913470101505174, 0.19187653242724867, 0.08815929268341582, 3.1499401611373434e-13, 2.3159264584293737e-05, 0.2207904294778822, 0.00028548883181989625, 0.16125908330785688, 2.34520953004691e-13, 0.07383911475103466, 1.261340744308434e-11, 0.1669310679761984, 5.887217796596289e-17, 0.004405800017178052, 0.026596210797298347, 0.015888106659506462, 1.4046923346722195e-15, 0.11626625143367586, 4.0609456119014564e-11, 1.3639784079977346e-07, 0.013089025444543934, 0.00023439687226645134, 3.402826400611403e-05, 0.05821534896261915, 1.5110159790054922e-09, 5.7302694522087805e-05, 0.2073126014528326, 9.688990494360894e-06, 0.19465120208527242, 3.2973106359559558e-09, 1.0866565869349643e-07, 0.025678546094557664, 2.3701554825988208e-08, 8.050579955280746e-10, 0.00048229436287305916, 1.3890109223403668e-12, 0.0004211018345534612, 0.19576128509628266, 0.06458458696260094, 0.0360317226804856, 0.0009245207885424378, 0.19707663292724797, 1.20160344341258e-06, 2.4214229335821e-10, 0.22522873509016916, 0.05453600143676085, 0.16679145495406122, 0.22016489036942688, 0.19172026538815357, 1.3837698890385719e-12, 0.028552829601964813, 0.17226184603976935, 1.998862662845255e-10, 0.16459025488347478, 0.00020996253747052052, 0.006821560754257059, 0.24980813542164038, 0.004079745921995323, 3.276170328948835e-11, 0.0006750257741399146, 0.0015517051045330632, 4.198484841411165e-05, 0.19129722646246816, 8.231578616513247e-05, 0.006821560754257059, 7.119213362610007e-08, 0.03869982202406019, 3.0487995483254038e-12, 0.008204427861253638, 1.515906744366008e-07, 0.09922458727756056, 0.053921118416417446, 0.018483119307968865, 0.00017643045751536413, 1.6217537886068813e-09, 2.921610639650249e-06, 0.21943362646121845, 0.08625540425304834, 0.2044783719998633, 5.449546085982242e-09, 0.0001566191191810784, 4.219228144164769e-09, 0.15994904289073908, 0.23043201210174535, 1.1065726979826371e-07, 1.9016157203045024e-09, 0.00016419311156812277, 2.7608416150026044e-11, 0.0011275533620227485, 0.08894932140056788, 2.08843938778909e-11, 6.547090846624037e-08, 0.17885280462122055, 8.975358289031732e-05, 0.05968021485210406, 0.0007917170260729467, 1.215718433804358e-07, 0.0010731333176385472, 1.2413469108239848e-07, 4.846728710646703e-10, 5.023539002856937e-10, 4.503897255148104e-13, 1.2393047411863797e-10, 0.005541604251648735, 0.00017105430346564532, 1.7909036845436947e-06, 1.415104584499751e-08, 2.049278005886439e-06, 0.00010229596550285423, 0.24992623080120602, 6.0768368161934205e-06, 3.0574939174884464e-06, 0.0024778223646872988, 0.1399617974999281, 2.5935436380741805e-06, 0.2243144146613414, 1.0216756921813813e-10, 0.008886399902167774, 0.00039663530256972557, 0.00013663221480730847, 0.001955598714995066, 0.0015056095034768902, 0.0025658105304724454, 0.16380345018887232, 0.24554646371441535, 8.553720634606795e-08, 0.00018461912728273688, 0.15848936122603496, 0.0009578118723272411, 1.8178626842220045e-07, 0.09336193158157635, 0.06987068349328851, 7.461463066328611e-11, 0.23100560023604988, 0.2747942275213564, 2.014001189686531e-06, 3.810134239904707e-10, 0.24574097729237065, 2.197191703023904e-06, 3.0623845635346925e-05, 5.033740013899758e-10, 2.5563864230766092e-11, 0.2076044761280298, 2.4083804613272974e-05, 2.3005185647087534e-11, 0.1809265588749538, 0.11314233971919196, 1.2912147252075457e-05, 0.2418288024595303, 0.00553526250094194, 8.922594023630381e-07, 0.08237142674468756, 0.06558132559480745, 0.0002287969446169185, 1.8545702831218591e-06, 0.00013311340107446945, 1.7878511141705168e-09, 1.2486258376223115e-07, 1.1657832011069402e-05, 0.32962173430917474, 2.569860948022952e-08, 1.7008475790974975e-14, 5.368431935971751e-13, 0.12196693027178673, 4.521385709643972e-07, 8.737701312272125e-11, 4.0948452750599884e-05, 3.547164019760893e-10, 5.393189150479372e-13, 0.00039672175195817997, 0.032799355525556795, 0.005077162554491421, 1.0330639762579953e-08, 9.003913638740016e-07, 1.467056761117469e-05, 4.763419703524008e-05, 1.2477924956663775e-06, 0.001461792231847392, 0.13926858842456702, 0.0019367460408901138, 2.9580959757212477e-08, 0.0428934775820633, 3.0393253506403456e-11, 1.5955119170296683e-15, 0.005620085675085429, 4.5789687227993335e-09, 0.000611007131786856, 0.00015933806244916582, 3.7045555776327177e-06, 0.21820729817083717, 0.04219768055630957, 0.005347432509094974, 5.033740013899758e-10, 2.877737300603471e-08, 0.0057999427348225385, 0.009959757892860965, 0.23193400581041956, 8.933380287102404e-08, 4.465519924320982e-11, 0.006831288830849935, 0.049678679273433834, 0.02805526939248426, 0.08619498592006258, 2.0549280818071836e-05, 0.1187975701181081, 0.12845593635773933, 3.9396597973934827e-10, 0.08051169896229382, 0.0001469507629932032, 0.07039560735052719, 0.1822665225743081, 0.018417526832961606, 0.19408931420024178, 0.21496161933189528, 0.0003055672994438652, 4.353613823615543e-15, 4.783429362787035e-09, 5.738543863188783e-12, 0.3800962693072949, 0.0007856497305098742, 1.5161683531925152e-09, 1.7697958305020736e-07, 5.59612631608657e-09, 0.15764086868227944, 6.726173633931102e-07, 0.0392809460287209, 0.010623713875516527, 0.02258954953866965, 2.3658204628533054e-07, 6.177501881365899e-06, 3.2945450434418433e-06, 0.00585740740637226, 0.02165066919491121, 0.00864626041376848, 4.746523630660016e-08, 0.0009152834772150342, 0.000331367649489378, 0.00045843785437615914, 0.24020614408546948, 1.873494665083624e-05, 2.290298159457471e-12, 1.6550668946241007e-10, 0.06375258456916222, 5.016035662367855e-07, 1.1718063896104525e-05, 0.144165170239134, 0.23841421906495097, 0.00011972330543619236, 0.002858616115853252, 2.0461510034541774e-05, 2.460791836988996e-05, 0.02281430613255897, 0.08556604856508976, 8.466813484098168e-06, 0.00028058491755575193, 1.9734490847467256e-05, 1.6422790860462066e-08, 0.02514520551717503, 0.016612830585848393, 2.613436810711134e-14, 5.241413433778702e-05, 2.2936002598041016e-10, 0.22980869485239852, 0.10914387137953696, 2.142718890773415e-12, 2.4762748755513542e-05, 4.281215771173683e-07, 1.6789381331064291e-06, 1.5821107390758676e-11, 0.013456782419272287, 0.23835901766572043, 8.476377363816076e-08, 0.09865429801493497, 0.18959195340003365, 4.727808026799493e-11, 4.444553111839682e-06, 3.3466806931688143e-06, 8.593821254621048e-08, 0.013301923079646505, 5.86418946372071e-09, 0.24517511802545444, 0.18969844236350952, 1.7367509414813187e-07, 0.07878664782874534, 8.727916460035939e-09, 8.787925372584775e-07, 2.972032017191373e-11, 6.227268594085809e-07, 1.3272684004379115e-10, 1.5100845353354687e-09, 0.0004156332635432136, 0.2445814724630413, 0.19200908652939952, 0.009125334283571984, 0.03661811961094452, 0.04293038158573825, 2.6220359461426312e-06, 9.212592159801858e-06, 0.12463071431539019, 2.2616274087880472e-12, 0.006167013668317622, 0.00027281377365717674, 4.4100256215914846e-12, 2.6903089077236902e-11, 3.402949627506757e-05, 8.767705417483777e-10, 0.010421039524723952, 2.4961710163529105e-05, 0.04773038406587382, 3.828697341806686e-15, 0.07214951754985588, 8.534794517173552e-10, 0.0011402512027156586, 0.007933058242145297, 2.312141604207934e-09, 8.809495017150781e-14, 0.22109950755281438, 0.0010446160274524274, 2.2198715635292745e-07, 1.2804265026727935e-14, 0.00038176102475762086, 9.974015916566076e-12, 0.004981973525751249, 0.03890744312388339, 0.14392403583815971, 4.312658053481737e-10, 2.1626339998317978e-10, 2.418809510323916e-13, 0.014693357011242192, 0.24584692344612735, 0.0019130034381831251, 8.898560156512777e-07, 0.007997437789595274, 0.00029177879063180256, 0.24979332714986047, 0.010475626772674218, 0.03934570822786676, 2.142656397360193e-11, 0.1937261308365701, 1.9150858121541603e-06, 1.7128601891832967e-07, 1.016154415056647e-06, 0.017432123120291854, 7.214325843385438e-07, 5.84715652632295e-15, 2.9179620097963266e-07, 6.75936681843528e-11, 0.02498564978876061, 0.14729608708001596, 0.03368748867466409, 1.0317473301315217e-05, 1.2003255568780399e-05, 1.2645250902865255e-08, 0.06944358960528874, 0.014031543755359088, 0.0016688724377855572, 1.130077449803743e-11, 0.03997167951676861, 0.031069020609627795, 0.03091062681639234, 0.0008665837299437484, 0.003627111365584801, 0.02258230303523323, 0.04843511588598659, 1.0651109368264855e-05, 6.065203089517603e-05, 0.23893908435895028, 0.05818053685896718, 0.23878561052588565, 0.011175955397176196, 2.8213689138745224e-05, 5.68519197732386e-08, 0.00040370784410281857, 0.1374473128001934, 0.008010032409544898, 4.319973030519897e-05, 0.24976662141452888, 0.08254446376018887, 4.379001266796578e-05, 0.13162172101576397, 2.929770111031408e-09, 2.6964023687777307e-06, 2.916899359162173e-06, 0.03411736539558574, 0.18837254024807093, 0.08858571494306489, 0.12321905387031262, 5.682884323115621e-07, 4.337893018229984e-07, 0.037705290346225384, 0.14856327479888787, 6.994160364204305e-07, 0.0009340928106642554, 0.00034922766791332786, 2.0497650285260498e-07, 1.1930902911776653e-06, 0.0026808847912374034, 0.00962489169066938, 3.768756639773753e-09, 0.23628289779350453, 0.06419615451842753, 0.00594000007612383, 0.11791591864030115, 0.08201160921191497, 0.062452598800598835, 7.30582586615236e-14, 0.010863685865658113, 2.1980264191719123e-05, 1.325726312582931e-12, 1.764825230399225e-10, 8.604173154123582e-10, 1.7338031594649195e-05, 1.233378498569673e-05, 0.028579149286781513, 2.8292935431561278e-11, 0.2470997176318548, 8.859481392953228e-08, 0.0243764397081081, 1.235837820967589e-12, 0.0729168136884224, 0.14453952998943476, 0.04458069298381151, 0.06617502638681572, 4.472847451086892e-06, 1.526392756721201e-08, 0.042393878597053014, 1.7240495055886873e-14, 0.0320692311147841, 6.561735532968713e-07, 0.08338103891673065, 7.247440616717747e-09, 0.22652118795134135, 3.535251933417527e-07, 1.5009013977951464e-09, 5.581109937539594e-07, 8.246717129845038e-06, 0.1025006684359222, 2.2098629193071685e-07, 1.532658928157099e-12, 9.161280328101595e-11, 2.952190358493367e-14, 7.084435331775052e-09, 0.09604824209642793, 0.024590151818722594, 0.020201223124462375, 0.2089088368248455, 0.07914551338017728, 1.4064905771348163e-07, 2.2867535046989552e-07, 0.24534406866807368, 0.012003193685915292, 0.17353730651516752, 0.2242494889646982, 4.2715351763512877e-11, 0.06247005989182264, 1.659164734135743e-05, 5.321494763940726e-12, 4.3614746858988696e-05, 1.705965175981955e-12, 0.22029798973560277, 4.752922509596502e-08, 1.0310646652023081e-05, 8.085995499920468e-08, 0.18973158586419103, 3.0221721776912958e-05, 0.14065950069874922, 2.732984364425123e-09, 0.012989987552053828, 2.514153521216691e-07, 0.24199485251585617, 0.24511232191740362, 8.240186003671823e-05, 0.0022659427550628664, 0.0341211582076264, 4.399600405763067e-08, 1.9965479370644135e-07, 0.23380049462514485, 0.22473105282354378, 0.04408617900136935, 0.21406986430302144, 0.17724093746061476, 0.16654229195185688, 0.0016666550537292046, 0.1828717706423384, 0.00012394275079530281, 2.8280343073849162e-11, 0.10370652199793562, 0.0006948570514591079, 2.0962129798889756e-09, 9.064023545612976e-08, 0.0026580013975081584, 3.999513201802252e-05, 0.027397518949035193, 0.00833139194902599]

In [14]:
matrix_var = diagonal_matrix(list_var)*len(list_var)

In [ ]:
(matrix_var.det()).n()



In [ ]:
1

In [2]:
def estimate_SCA_prob(report_every, dbdd, measured, max_guesses, D_s, to_print):
    #Upper bounding the maximum number of guessed coordinate
    """ 
    This function evaluates the security loss after Bos et al attack
    :report_every: an integer that give the frequency of
    logging (None for no logging)
    :dbdd: instance of the class DBDD
    :measured: table representing the (simulated) information
    given by Bos et al attack
    :max_guesses: integer for upperbounding the number of guesses
    """
    add_poster = True
    MINIMUM_VAR = 0.01
    proba_best = {}
    Id = identity_matrix(n + m)
    list_av = [0] * n
    list_var = [0] * n
    _, s_s = average_variance(D_s)
    intersect_indices = []
    for i in range(n):
        apost_dist = measurement_to_aposteriori(measured[i])
        if apost_dist is not None:
            proba_best[i] = max([apost_dist[j] for j in apost_dist.keys()])
            av, var = average_variance(apost_dist)
            av = float(av)
            var = float(var)
            list_av[i] = av
            list_var[i] = var
            if av*av > s_s - var:
                intersect_indices += [i]
        else:
            av = None
            var = None
        v = vec(Id[i])
        if report_every is not None and ((i % report_every == 0) or (i == n - 1)) :
            verbose = 2 
        else:
            verbose = 0
        dbdd.verbosity = verbose
        if verbose == 2:
            logging("[...%d]" % report_every, newline=False)
        if var is None:
            logging("   alert! var is None     ", style="HEADER")
        if var == 0:
            logging("     Real Perfect Hint /////    ", style="HEADER")
            list_var[i] = MINIMUM_VAR
            
    if to_print:    
        print("number of elements in intersect_indices", len(intersect_indices))
        
    # Changes to intersect_indices
    sigma_1_squared = s_s
    sigma_2_squared = list_var
    cs = list_av
    alphas = [-1]*n
    betas = [-1]*n
    for i in range(n):
        if sigma_1_squared > sigma_2_squared[i] and sigma_2_squared[i] != 0:
            alphas[i] = (cs[i])**2 / (sigma_1_squared - sigma_2_squared[i])
            betas[i] = sigma_1_squared / sigma_2_squared[i]
    
    intersect_indices_optimized = find_optimal_indices(alphas, betas, cs, sigma_1_squared, m)

    # if to_print:
    #     print("--comparing real secret v.s. SCA distribution--")
    #     for i in range(len(list_av)):
    #         print(list_av[i], list_var[i], dbdd.u[0, m+i])
    #     print("--finish comparing--")
    #####Intersection Phase####
#     dbdd.S, intersect_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n, intersect_indices, to_print)
    if not add_poster:
        dbdd.S, intersect_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n, intersect_indices, intersect_indices_optimized, to_print)
        dbdd.mu = intersect_mu
    else:
        intersect_S, intersect_mu, poster_S, poster_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n, intersect_indices, intersect_indices_optimized, to_print)
        #integrating SCA through posteriori distribution
        dbdd.S = poster_S
        dbdd.mu = poster_mu
        if to_print:
            (poster_beta, _) = dbdd.estimate_attack()
            print("poster_beta", poster_beta)
            # dbdd.integrate_q_vectors(q, indices=range(n, n + m))
            # (poster_beta_after_q, _) = dbdd.estimate_attack()
            # print("poster_beta_after_q: ", poster_beta_after_q)
        
        dbdd.S = intersect_S
        dbdd.mu = intersect_mu
        # if to_print:
        #     print("after intersection, dbdd.u: ", RR(dbdd.mu)[0, -5:])
    

    if to_print:
        print("ellip_norm after intersection: ")

        # print((dbdd.u - dbdd.mu).nrows(), (dbdd.u - dbdd.mu).ncols())
        # print((matrix(np_inv(diagonal_matrix(dbdd.S)))).nrows(), (matrix(np_inv(diagonal_matrix(dbdd.S)))).ncols())  
        print(scal((dbdd.u - dbdd.mu) * matrix(np_inv(diagonal_matrix(dbdd.S)))/(n + m + 1) * (dbdd.u - dbdd.mu).T)) #should be <=1
    
    (beta, _) = dbdd.estimate_attack()
    if to_print:
        print("intersect_beta: ", beta)
        # dbdd.integrate_q_vectors(q, indices=range(n, n + m))
        # (intersect_beta_after_q, _) = dbdd.estimate_attack()
        # print("intersect_beta_after_q: ", intersect_beta_after_q)
    
    #####Guess phase#####
    
    if report_every is not None:
        logging("     Hybrid attack estimation     ", style="HEADER")
    
    intersect_var = list(dbdd.S)
    intersect_av = list(dbdd.mu[0])
    proba_best2 = {}
    for k in range(len(intersect_var)):
        var = intersect_var[k]
        if var is None:
            logging("   alert! var is None     ", style="HEADER")
            continue
        if var <= 0 + 0.02:
            v = vec(Id[k])
            logging("     Real Perfect Hint in Hybrid Phase    ", style="HEADER")
            intersect_var[k] = MINIMUM_VAR
            dbdd.S[k] = MINIMUM_VAR
            dbdd.integrate_perfect_hint(v, av, estimate=verbose)
        # print("k = ", k,intersect_var[k])
        proba_best2[k] = var_to_prob(intersect_var[k], intersect_av[k], D_s, intersect_av[k])
          
    sorted_guesses = sorted(proba_best2.items(),
                            key=lambda kv: - kv[1])

    sorted_guesses = [sorted_guesses[i][0] for i in range(len(sorted_guesses))]

   
    proba_success = 1.
    dbdd.verbosity = 0
    guesses = 0
    j = 0
    
    for t in sorted_guesses:
        j += 1
        if proba_success*proba_best2[t] >= max_guesses: #(guesses <= max_guesses):(guesses <= max_guesses):

            v = vec(Id[t])
            if dbdd.integrate_perfect_hint(v, _, force = False):
                guesses += 1
                proba_success *= proba_best2[t]

    beta_before_last_q = dbdd.beta
    if report_every is not None:
        dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
    else:
        dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
    return beta, dbdd.beta, proba_success, guesses, beta_before_last_q


In [5]:
# def reconstruct_diagonal_S(list_av, list_var, D_s, D_e, m, n, intersect_indices, to_print):
##Selected indices
    
#     mu_e, s_e = average_variance(D_e)
#     mu_s, s_s = average_variance(D_s)
    
#     #ellipsoids of first n dimension, rescaling to <= 1
#     Sigma_SCA = diagonal_matrix(list_var)*n
#     mu_SCA = vec(list_av)
#     Sigma_LWE = matrix(Sigma_SCA)*n
#     mu_LWE = matrix(mu_SCA)
   
#     for i in intersect_indices:
#         Sigma_LWE[i, i] = s_s*n
#         mu_LWE[0, i] = mu_s
        
      
    
#     logging("order B: input1 SCA ellip, input2 LWE ellip", style="HEADER")
#     ellipsoid_intersection(mu_SCA , Sigma_SCA, mu_LWE, Sigma_LWE, tolerance=1.48e-08)
#     logging("order A: input1 LWE ellip, intput2 SCA ellip", style="HEADER")
#     new_mu_sub, new_S_sub = ellipsoid_intersection(mu_LWE, Sigma_LWE, mu_SCA , Sigma_SCA, tolerance=1.48e-08)
    
#     new_S1 = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), new_S_sub)
#     new_S = block4(new_S1, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1))
#     rest_S = diagonal_matrix(m * [s_e] + n * [0] + [1])
    
#     hybrid_S = (new_S/n + rest_S)
#     diagonal_S = np.array([RR(hybrid_S[i, i]) for i in range(hybrid_S.nrows())])
    
#     new_mu = concatenate([new_mu_sub, [mu_e]*m + [0]])
#     return diagonal_S, new_mu
def reconstruct_diagonal_S(list_av, list_var, D_s, D_e, m, n, intersect_indices, intersect_indices_optimized, to_print):
    ##full n submatrix
    full = False
    add_poster = True
    if full:
        mu_SCA = vec(list_av)
        S_SCA = diagonal_matrix(list_var)*(n)
        
        mu_e, s_e = average_variance(D_e)
        mu_s, s_s = average_variance(D_s)
        mu_LWE = vec(n * [mu_s])
        S_LWE = diagonal_matrix(n * [s_s])*(n)
    
    else:

        ####partial coordinates###
        mu_s, s_s = average_variance(D_s)
        mu_e, s_e = average_variance(D_e)
        
        #ellipsoids of first n dimension, rescaling to <= 1
        S_SCA = diagonal_matrix(list_var)*n
        # if to_print:
            # print("1. list_var", list_var)
            # print("1. ln det(SCA matrix): ", logdet(S_SCA).n())#(S_SCA.det()).n(), ln(S_SCA.det()).n())
        mu_SCA = vec(list_av)
        S_LWE = matrix(S_SCA)
        mu_LWE = matrix(mu_SCA)
        S_LWE_2 = matrix(S_SCA)
        mu_LWE_2 = matrix(mu_SCA)

        for i in intersect_indices:
            # if to_print:
            #     print("ebdd.S should be close to Sigma_SCA")
            #     print("s_s, ebdd.S (rescaling to <= 1", s_s, ebdd.S[i,i]*n)
            #     assert(ebdd.S[i,i]*n - Sigma_SCA[i,i] < 10**-5)
            S_LWE[i, i] = s_s*n #<=1
            mu_LWE[0, i] = mu_s
            
        for i in intersect_indices_optimized:
            # if to_print:
            #     print("ebdd.S should be close to Sigma_SCA")
            #     print("s_s, ebdd.S (rescaling to <= 1", s_s, ebdd.S[i,i]*n)
            #     assert(ebdd.S[i,i]*n - Sigma_SCA[i,i] < 10**-5)
            S_LWE_2[i, i] = s_s*n #<=1
            mu_LWE_2[0, i] = mu_s
        ##########################
        if to_print:
            print("1. ln det(LWE matrix): ", logdet(S_LWE))#(S_LWE.det()).n(), ln(S_LWE.det()).n())
            print("1. ln det(SCA matrix) - ln det(LWE matrix)", logdet(S_SCA) - logdet(S_LWE))#(S_SCA.det()).n() / (S_LWE.det()).n(), ln(S_SCA.det()).n() - ln(S_LWE.det()).n())

    if to_print:
        partial_u = vec(dbdd.u[0, n:n+m])

        #S_SCA_full:
        S_SCA_SE = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), S_SCA) #<=1
        S_SCA_SE1 = block4(S_SCA_SE, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1)) #<=1
        rest_S_SCA = diagonal_matrix(m * [s_e] + n * [0] + [1]) #<=m
    
        S_SCA_full = (S_SCA_SE1/n + rest_S_SCA) #<=n+m+1
        #mu_SCA_full
        mu_SCA_full = concatenate([[mu_e]*m, mu_SCA, [0]])

        #S_LWE_full:
        S_LWE_SE = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), S_LWE) #<=1
        S_LWE_SE1 = block4(S_LWE_SE, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1)) #<=1
        rest_S_LWE = diagonal_matrix(m * [s_e] + n * [0] + [1]) #<=m
    
        S_LWE_full = (S_LWE_SE1/n + rest_S_LWE) #<=n+m+1
        #mu_LWE_full
        mu_LWE_full = concatenate([[mu_e]*m, mu_LWE, [0]])
        print("partial ellip_norm of input1 SCA ellip (dim = n): ")
        print(scal((partial_u - mu_SCA) * matrix(np_inv(S_SCA)) * (partial_u - mu_SCA).T)) #should be <=1

        print("full ellip_norm of input1 SCA ellip (dim = n + m + 1, should be <=1): ")
        print(scal((dbdd.u - mu_SCA_full) * matrix(np_inv(S_SCA_full))/(n + m + 1) * (dbdd.u - mu_SCA_full).T)) #should be <=1

        print("partial ellip_norm of input1 LWE ellip (dim = n): ")
        print(scal((partial_u - mu_LWE) * matrix(np_inv(S_LWE)) * (partial_u - mu_LWE).T)) #should be <=1
        print("full ellip_norm of input1 LWE ellip  (dim = n + m + 1, should be <=1): ")
        print(scal((dbdd.u - mu_LWE_full) * matrix(np_inv(S_LWE_full))/(n + m + 1) * (dbdd.u - mu_LWE_full).T)) #should be <=1
    if to_print:
        print("ln det(SCA matrix): ", logdet(S_SCA))#(S_SCA.det()).n(), ln(S_SCA.det()).n())
        print("ln det(LWE matrix): ", logdet(S_LWE))#(S_LWE.det()).n(), ln(S_LWE.det()).n())
        print("ln det(SCA matrix) - ln det(LWE matrix)", logdet(S_SCA) - logdet(S_LWE))#(S_SCA.det()).n() / (S_LWE.det()).n(), ln(S_SCA.det()).n() - ln(S_LWE.det()).n())
        print("ln det(LWE_opt matrix): ", logdet(S_LWE_2))#(S_LWE.det()).n(), ln(S_LWE.det()).n())
        print("ln det(SCA matrix) - ln det(LWE_opt matrix)", logdet(S_SCA) - logdet(S_LWE_2))#(S_SCA.det()).n() / (S_LWE_2.det()).n(), ln(S_SCA.det()).n() - ln(S_LWE_2.det()).n())
    logging("order B: input1 SCA ellip, input2 LWE ellip", style="HEADER")
    ellipsoid_intersection(mu_SCA, S_SCA, mu_LWE, S_LWE, tolerance=1.48e-08)
    logging("order A: input1 LWE ellip, intput2 SCA ellip", style="HEADER")
    
    if to_print:
        import time
        start = time.time()
    new_mu_sub, new_S_sub = ellipsoid_intersection(mu_LWE, S_LWE, mu_SCA, S_SCA, tolerance=1.48e-08) #<=1
    if to_print:
        end = time.time()
        print("total time for ellipsoid_intersection:", end - start)
    # if to_print:
    #     print("matrix after intersection: ", new_S_sub[:5,:5])
    if to_print:
        print("ln det(intersected matrix): ",  logdet(new_S_sub))
        
    logging("order B: input1 SCA ellip, input2 LWE_opt ellip", style="HEADER")
    ellipsoid_intersection(mu_SCA, S_SCA, mu_LWE_2, S_LWE_2, tolerance=1.48e-08)
    logging("order A: input1 LWE_opt ellip, intput2 SCA ellip", style="HEADER")
    
    if to_print:
        import time
        start = time.time()
    new_mu_sub_2, new_S_sub_2 = ellipsoid_intersection(mu_LWE_2, S_LWE_2, mu_SCA, S_SCA, tolerance=1.48e-08) #<=1
    if to_print:
        end = time.time()
        print("total time for ellipsoid_intersection:", end - start)
    # if to_print:
    #     print("matrix after intersection: ", new_S_sub[:5,:5])
    if to_print:
        print("ln det(intersected matrix): ",  logdet(new_S_sub_2))
    
    if logdet(new_S_sub_2) < logdet(new_S_sub):
        new_mu_sub = new_mu_sub_2
        new_S_sub = new_S_sub_2
    
    new_S1 = block4(diagonal_matrix([0] * m), zero_matrix(ZZ, m, n), zero_matrix(ZZ, n, m), new_S_sub) #<=1
    new_S = block4(new_S1, zero_matrix(ZZ, n + m, 1), zero_matrix(ZZ, 1, n + m), zero_matrix(ZZ, 1, 1)) #<=1
    rest_S = diagonal_matrix(m * [s_e] + n * [0] + [1]) #<=m
    
    hybrid_S = (new_S/n + rest_S) #<=n+m+1
    diagonal_S = np.array([RR(hybrid_S[i, i]) for i in range(hybrid_S.nrows())])
    # if to_print:
    #     print(diagonal_S[n:n+5])
    if add_poster:
        diagonal_SCA = np.array([RR(S_SCA_full[i, i]) for i in range(hybrid_S.nrows())])
    new_mu = concatenate([[mu_e]*m, new_mu_sub, [0]])
    return diagonal_S, new_mu, diagonal_SCA, mu_SCA_full
def var_to_prob(var, mean, D_s, x):
    discrete_prob_list = [gaussian_prob(var, mean, x) for x in D_s.keys()]
#     print(discrete_prob_list)
    norm = sum(discrete_prob_list)

    denom = sqrt(var * 2 * pi)
    ex = exp(-0.5 * (x - mean)**2 / var)
    return ex/denom/norm
def gaussian_prob(var, mean, x):
    denom = sqrt(var * 2 * pi)
    ex = exp(-0.5 * (x - mean)**2 / var)
    # print("in gaussian_prob: ", var, mean, x, denom, ex)
    return 1/denom * ex

In [ ]:
# def integrate_posteriori_dist(ebdd, A, b, q, list_av, list_var, D_e):
# #   needs_ellipsoid_scaling(scale=False)
#     print("scaled@funct posteriori_integration? suppose False: ", ebdd.scaled)
    
#     m = A.nrows()
#     n = A.ncols()
#     mu_e, s_e = average_variance(D_e)
    
#     mu_s_post = vec(list_av)
    
#     #Update mu
#     ebdd.mu = kannan_ellipsoid_mean_update(A, b, mean_s = mu_s_post, mean_e = vec([mu_e]*m))
    
#     #Update S
#     #1. Compute Sigma_s_e
#     zero1 = zero_matrix(ZZ, m, n)
#     Sigma_s_e = diagonal_matrix(list_var + [s_e]*m)
#     #2. Compute S
#     upleft_mat_inv = block4(identity_matrix(n), A.T/q, zero1, 1/q*identity_matrix(m)) #upleft_mat.inverse()
#     ebdd.S = upleft_mat_inv.T * Sigma_s_e * upleft_mat_inv    

#     return

In [4]:
def data_to_measure(sca_seeds, params, param, frodo_distribution): 
    # Reading the score tables from Bos et al. attack
    scores = []
    correct = []
    for seed in sca_seeds:
        for i in range(1, 9):
            data = loadmat('Scores_tables_SCA/' + params +
                           '/apostandcorrect' + str(param + 1) +
                           '_seed' + str(seed) +
                           'nbar' + str(i) + '.mat')
            scores += list(data['apostdist'])
            correct += list(data['correct'])
    measures = {}
    # the score tables are stored according to the secret coefficient. We use them
    # for generating the measurement.
    for key_guess in range(-len(frodo_distribution)+1, len(frodo_distribution)):
        measures[key_guess] = [scores[ind] for ind in
                  [i for i, d in enumerate(correct) if
                   recenter(d) == key_guess]]
    return measures
def simu_measured(secret, measures):
    """
    This fonction simulates the information gained by
    Bos et al attack. For a given secret, the simulation
    is outputting a score table at random from the data of Bos et al.
    :secret: an integer being the secret value
    :measurement: a score table
    """
    if secret in measures.keys():
        measurement = random.choice(measures[secret])
    else: 
        measurement = None
    return measurement

def measurement_to_aposteriori(measurement):
    """
    This fonction transforms a score table into an aposteriori distribution.
    According to the matlab code of Bos et al. attack,
    the score table is proportional
    to the logarithm of the aposteriori probability. We thus apply the exponential
    and renormalize.
    :measurement: a score table
    :apost_dist: a dictionnary of the aposteriori distribution
    """
    if measurement is not None:
        s = sum([exp(meas) for meas in measurement])
        measurement = [exp(meas)/s for meas in measurement]
        apost_dist = {}
        for key_guess in range(-len(frodo_distribution) + 1, len(frodo_distribution)):
            apost_dist[key_guess] = measurement[key_guess + len(frodo_distribution) - 1]
    else:
        apost_dist = None
    return apost_dist


In [61]:
#num
def estimate_SCA_num(report_every, dbdd, measured, max_guesses, D_s, to_print):
    #Upper bounding the maximum number of guessed coordinate
    """ 
    This function evaluates the security loss after Bos et al attack
    :report_every: an integer that give the frequency of
    logging (None for no logging)
    :dbdd: instance of the class DBDD
    :measured: table representing the (simulated) information
    given by Bos et al attack
    :max_guesses: integer for upperbounding the number of guesses
    """
    MINIMUM_VAR = 0.01
    proba_best = {}
    Id = identity_matrix(n + m)
    list_av = [0] * n
    list_var = [0] * n
    
    _, s_s = average_variance(D_s)
    intersect_indices = []
    
    for i in range(n):
        apost_dist = measurement_to_aposteriori(measured[i])
        if apost_dist is not None:
            proba_best[i] = max([apost_dist[j] for j in apost_dist.keys()])
            av, var = average_variance(apost_dist)
            av = float(av)
            var = float(var)
            list_av[i] = av
            list_var[i] = var
            if av*av > s_s - var:
                intersect_indices += [i]
        else:
            av = None
            var = None
        v = vec(Id[i])
        if report_every is not None and ((i % report_every == 0) or (i == n - 1)) :
            verbose = 2 
        else:
            verbose = 0
        dbdd.verbosity = verbose
        if verbose == 2:
            logging("[...%d]" % report_every, newline=False)
        if var is None:
            logging("   alert! var is None     ", style="HEADER")
        if var == 0:
            logging("     Real Perfect Hint     ", style="HEADER")
            list_var[i] = MINIMUM_VAR

    dbdd.S, intersect_mu = reconstruct_diagonal_S(list_av, list_var, D_s, D_s, m, n, None, to_print)
    dbdd.mu = intersect_mu
    
    (beta, _) = dbdd.estimate_attack()
    
    #####Guess phase#####
    if report_every is not None:
        logging("     Hybrid attack estimation     ", style="HEADER")
    
    intersect_var = list(dbdd.S) #only for DBDD_diag case
    intersect_av = list(intersect_mu[0])
    proba_best2 = {}
    for k in range(len(intersect_var)):

        if var is None:
            logging("   alert! var is None     ", style="HEADER")
            continue
        if var <= 0 + 0.02:
            v = vec(Id[k])
            logging("     Nooo Real Perfect Hint     ", style="HEADER")

            intersect_var[k] = MINIMUM_VAR
            dbdd.S[k] = MINIMUM_VAR
            dbdd.integrate_perfect_hint(v, av, estimate=verbose)
        proba_best2[k] = var_to_prob(intersect_var[k], intersect_av[k], D_s, intersect_av[k])


    sorted_guesses = sorted(proba_best2.items(),
                            key=lambda kv: - kv[1])

    sorted_guesses = [sorted_guesses[i][0] for i in range(len(sorted_guesses))]

    proba_success = 1.
    dbdd.verbosity = 0
    guesses = 0
    j = 0
    for t in sorted_guesses:
        j += 1
        if (guesses < max_guesses):

            v = vec(Id[t])
            if dbdd.integrate_perfect_hint(v, _, force = False):
                guesses += 1
                proba_success *= proba_best2[t]
    beta_before_last_q = dbdd.beta
    print("before last q_vector", beta_before_last_q)
    if report_every is not None:|
        dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m), report_every=report_every)
    else:
        dbdd.integrate_q_vectors(q, min_dim = n + 1, indices=range(n, n + m))
    return beta, dbdd.beta, proba_success, guesses, beta_before_last_q
